<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Modelo de lenguaje con tokenización por caracteres

### Consigna
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.


### Sugerencias
- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.


In [34]:
import os
import random


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import seaborn as sns

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Usaremos las utilidades de procesamiento de textos y secuencias de Keras
from tensorflow.keras.preprocessing.text import Tokenizer # equivalente a ltokenizer de nltk
from tensorflow.keras.preprocessing.text import text_to_word_sequence # equivalente a word_tokenize de nltk
from tensorflow.keras.utils import pad_sequences # se utilizará para padding

from keras.layers import Input, TimeDistributed, CategoryEncoding, SimpleRNN, Dense
from keras.models import Model, Sequential


In [38]:
RANDOM_SEED = 5440
BATCH_SIZE = 256

Así como lo hicimos en el desafío anterior, buscamos las diferentes recetas del mismo. Para simplificar el desarrollo del TP, voy a replicar algunas funciones, aunque la idea en un entorno producitvo es tener locaciones comunes, librerías estandars de recupero y manipulación de la información.

In [2]:
def get_recipes_files(folder_paths):
    files = []
    for folder_path in folder_paths:
        files.extend([os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.txt') or file.endswith('.md')])
    return files

In [6]:
folder_paths = ['../Desafio 2/recipes_1', '../Desafio 2/recipes_2']
all_files = get_recipes_files(folder_paths)
print("All files:")
print(all_files)

All files:
['../Desafio 2/recipes_1/almonds.txt', '../Desafio 2/recipes_1/fava.txt', '../Desafio 2/recipes_1/squares.txt', '../Desafio 2/recipes_1/recipes2.txt', '../Desafio 2/recipes_1/brownies.txt', '../Desafio 2/recipes_1/pannacotta.txt', '../Desafio 2/recipes_1/bruschetta.txt', '../Desafio 2/recipes_1/sweet_potato_pie.txt', '../Desafio 2/recipes_1/sorbet.txt', '../Desafio 2/recipes_1/cauli.txt', '../Desafio 2/recipes_1/tart.txt', '../Desafio 2/recipes_1/cornbread.txt', '../Desafio 2/recipes_2/crispy-beef-with-egg-fried-rice.md', '../Desafio 2/recipes_2/classic-duck-breast.md', '../Desafio 2/recipes_2/crispy-sesame-chicken.md', '../Desafio 2/recipes_2/tomato-pasta.md', '../Desafio 2/recipes_2/beef-stroganoff.md', '../Desafio 2/recipes_2/carrot-cake.md', '../Desafio 2/recipes_2/cacio-e-peppe.md', '../Desafio 2/recipes_2/pizza-sauce.md', '../Desafio 2/recipes_2/tagliatelle-with-broccoli-cauliflower-and-blue-cheese.md', '../Desafio 2/recipes_2/pizza-dogs.md', '../Desafio 2/recipes_2/me

In [7]:
# El encoding latin-1 es necesario para leer los archivos de texto. Si no se especifica, se produce un error de UnicodeDecodeError:
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 2: invalid continuation byte
df_recipes = pd.DataFrame({'recipe': [open(file, encoding='latin-1').read() for file in all_files]})

In [8]:
print("Cantidad de documentos:", df_recipes.shape[0])

Cantidad de documentos: 80


In [9]:
df_recipes

,recipe
0,"BLANCHING NUTS: In the case of nuts, especiall..."
1,(fava beans)\n\ntreat gently when young and fr...
2,"These are kid tested, mother approved. From Co..."
3,\nless.\n\n1 cup dried split peas 1/2 teaspoon...
4,Fooder's Brownies (I usually 1/2 the recipe to...
...,...
75,---\ntitle: Beef and Guinness stew\ndate: 2021...
76,---\ntitle: Margherita pizza\ndate: 2021-07-18...
77,---\ntitle: Whisky Haggis sauce\ndate: 2023-01...
78,---\ntitle: Bosnian Stuffed Peppers (Punjene P...


In [13]:
def concatenate_recipes(df):
    """
    Concatena todas las recetas en un solo string.
    """
    return " ".join(df['recipe'].tolist())

# Concatenar todas las recetas
all_recipes_text = concatenate_recipes(df_recipes)
print(all_recipes_text[:1000])

BLANCHING NUTS: In the case of nuts, especially with almonds, pistachios 
and hazelnuts, in addition to the tough outer husk, they have a thin inner 
lining which needs to be removed. This lining can be bitter and somewhat 
unattractive when found in confection and baked goods. Blanching can be a 
time consuming process. Think about  purchasing them already blanched and 
skinned. Nuts can then be chopped and then toasted, if desired.

ALMONDS:	
Shell almonds and place them in a saucepan. Cover them with water.  

Boil for 2 to 3 minutes.  Drain and rinse nuts in cold water.  

Pinch off the almond skins by holding them at one end with your 
index finger and thumb, and then while pointing them into a bowl, press 
your fingers together -- be careful because they can shoot across the room!  
	
Place blanched nuts on a jelly roll or rimmed pan and dry them out 
in a 300 degree preheated oven for about 5 - 10 minutes. Don't let them 
brown.

----------

                    *  Exported from 

### Elegir el tamaño del contexto

En este caso, como el modelo de lenguaje es por caracteres, todo un gran corpus
de texto puede ser considerado un documento en sí mismo y el tamaño de contexto
puede ser elegido con más libertad en comparación a un modelo de lenguaje tokenizado por palabras y dividido en documentos más acotados.

In [10]:
# seleccionamos el tamaño de contexto
max_context_size = 100

In [15]:
chars_vocab = set(all_recipes_text)

In [16]:
# la longitud de vocabulario de caracteres es:
len(chars_vocab)

139

In [17]:
# Construimos los dicionarios que asignan índices a caracteres y viceversa.
# El diccionario `char2idx` servirá como tokenizador.
char2idx = {k: v for v,k in enumerate(chars_vocab)}
idx2char = {v: k for k,v in char2idx.items()}

## Tokenizar

In [22]:
tokenized_recipes_text = [char2idx[ch] for ch in all_recipes_text]

In [25]:
tokenized_recipes_text[:100]

[89,
 54,
 20,
 25,
 51,
 113,
 98,
 25,
 73,
 14,
 25,
 130,
 88,
 81,
 27,
 14,
 98,
 111,
 14,
 5,
 65,
 4,
 14,
 61,
 80,
 121,
 4,
 14,
 126,
 117,
 14,
 111,
 86,
 5,
 121,
 44,
 14,
 4,
 121,
 16,
 4,
 61,
 82,
 80,
 119,
 119,
 47,
 14,
 15,
 82,
 5,
 65,
 14,
 80,
 119,
 100,
 126,
 111,
 60,
 121,
 44,
 14,
 16,
 82,
 121,
 5,
 80,
 61,
 65,
 82,
 126,
 121,
 14,
 101,
 80,
 111,
 60,
 14,
 65,
 80,
 104,
 4,
 119,
 111,
 86,
 5,
 121,
 44,
 14,
 82,
 111,
 14,
 80,
 60,
 60,
 82,
 5,
 82,
 126,
 111]

### Organizando y estructurando el dataset

In [26]:
# separaremos el dataset entre entrenamiento y validación.
# `p_val` será la proporción del corpus que se reservará para validación
# `num_val` es la cantidad de secuencias de tamaño `max_context_size` que se usará en validación
p_val = 0.2
num_val = int(np.ceil(len(tokenized_recipes_text)*p_val/max_context_size))

In [27]:
# separamos la porción de texto utilizada en entrenamiento de la de validación.
train_text = tokenized_recipes_text[:-num_val*max_context_size]
val_text = tokenized_recipes_text[-num_val*max_context_size:]

In [28]:
tokenized_sentences_val = [val_text[init*max_context_size:init*(max_context_size+1)] for init in range(num_val)]

In [29]:
tokenized_sentences_train = [train_text[init:init+max_context_size] for init in range(len(train_text)-max_context_size+1)]

### Obtenemos el vector input y target

In [30]:
X = np.array(tokenized_sentences_train[:-1])
y = np.array(tokenized_sentences_train[1:])

Vamos a explorar un poco el input

In [31]:
X.shape

(178637, 100)

In [32]:
X[0,:10]

array([ 89,  54,  20,  25,  51, 113,  98,  25,  73,  14])

In [33]:
vocab_size = len(chars_vocab)

print(f"El tamaño del vocabulario es: {vocab_size}")

El tamaño del vocabulario es: 139


# Modelo

## 1. Definir el Modelo

El modelo que se propone como ejemplo consume los índices de los tokens y los transforma en vectores OHE (en este caso no entrenamos una capa de embedding para caracteres). Esa transformación se logra combinando las capas `CategoryEncoding` que transforma a índices a vectores OHE y `TimeDistributed` que aplica la capa a lo largo de la dimensión "temporal" de la secuencia.

In [35]:
model = Sequential()

model.add(TimeDistributed(CategoryEncoding(num_tokens=vocab_size, output_mode = "one_hot"),input_shape=(None,1)))
model.add(SimpleRNN(200, return_sequences=True, dropout=0.1, recurrent_dropout=0.1 ))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, None, 139)        0         
 ibuted)                                                         
                                                                 
 simple_rnn (SimpleRNN)      (None, None, 200)         68000     
                                                                 
 dense (Dense)               (None, None, 139)         27939     
                                                                 
Total params: 95,939
Trainable params: 95,939
Non-trainable params: 0
_________________________________________________________________


#### Vamos a construir la función de perplejidad mediante el uso de una función callback

In [36]:
class PplCallback(keras.callbacks.Callback):

    '''
    Este callback es una solución ad-hoc para calcular al final de cada epoch de
    entrenamiento la métrica de Perplejidad sobre un conjunto de datos de validación.
    La perplejidad es una métrica cuantitativa para evaluar la calidad de la generación de secuencias.
    Además implementa la finalización del entrenamiento (Early Stopping)
    si la perplejidad no mejora después de `patience` epochs.
    '''

    def __init__(self, val_data, history_ppl,patience=5):
      # El callback lo inicializamos con secuencias de validación sobre las cuales
      # mediremos la perplejidad
      self.val_data = val_data

      self.target = []
      self.padded = []

      count = 0
      self.info = []
      self.min_score = np.inf
      self.patience_counter = 0
      self.patience = patience

      # nos movemos en todas las secuencias de los datos de validación
      for seq in self.val_data:

        len_seq = len(seq)
        # armamos todas las subsecuencias
        subseq = [seq[:i] for i in range(1,len_seq)]
        self.target.extend([seq[i] for i in range(1,len_seq)])

        if len(subseq)!=0:

          self.padded.append(pad_sequences(subseq, maxlen=max_context_size, padding='pre'))

          self.info.append((count,count+len_seq))
          count += len_seq

      self.padded = np.vstack(self.padded)


    def on_epoch_end(self, epoch, logs=None):

        # en `scores` iremos guardando la perplejidad de cada secuencia
        scores = []

        predictions = self.model.predict(self.padded,verbose=0)

        # para cada secuencia de validación
        for start,end in self.info:

          # en `probs` iremos guardando las probabilidades de los términos target
          probs = [predictions[idx_seq,-1,idx_vocab] for idx_seq, idx_vocab in zip(range(start,end),self.target[start:end])]

          # calculamos la perplejidad por medio de logaritmos
          scores.append(np.exp(-np.sum(np.log(probs))/(end-start)))

        # promediamos todos los scores e imprimimos el valor promedio
        current_score = np.mean(scores)
        history_ppl.append(current_score)
        print(f'\n mean perplexity: {current_score} \n')

        # chequeamos si tenemos que detener el entrenamiento
        if current_score < self.min_score:
          self.min_score = current_score
          self.model.save("my_model")
          print("Saved new model!")
          self.patience_counter = 0
        else:
          self.patience_counter += 1
          if self.patience_counter == self.patience:
            print("Stopping training...")
            self.model.stop_training = True


## 2. Entrenar el modelo

In [ ]:
BATCH_SIZE = 256 if BATCH_SIZE is None else BATCH_SIZE

In [39]:
# fiteamos, nótese el agregado del callback con su inicialización. El batch_size lo podemos seleccionar a mano
# en general, lo mejor es escoger el batch más grande posible que minimice el tiempo de cada época.
# En la variable `history_ppl` se guardarán los valores de perplejidad para cada época.
history_ppl = []
hist = model.fit(X, y, epochs=20, callbacks=[PplCallback(tokenized_sentences_val,history_ppl)], batch_size=BATCH_SIZE)

Epoch 1/20
698/698 [==============================] - ETA: 0s - loss: 2.5381
 mean perplexity: 7.804616251749798 



INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


Saved new model!
698/698 [==============================] - 156s 223ms/step - loss: 2.5381
Epoch 2/20
698/698 [==============================] - ETA: 0s - loss: 2.0512
 mean perplexity: 6.370157874886093 



INFO:tensorflow:Assets written to: my_model/assets


...

INFO:tensorflow:Assets written to: my_model/assets


Saved new model!
698/698 [==============================] - 219s 314ms/step - loss: 1.5639
Epoch 13/20
698/698 [==============================] - ETA: 0s - loss: 1.5548
 mean perplexity: 4.439423524320347 



INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


Saved new model!
698/698 [==============================] - 212s 303ms/step - loss: 1.5548
Epoch 14/20
698/698 [==============================] - ETA: 0s - loss: 1.5470
 mean perplexity: 4.508191742048347 

698/698 [==============================] - 200s 287ms/step - loss: 1.5470
Epoch 15/20
698/698 [==============================] - ETA: 0s - loss: 1.5405
 mean perplexity: 4.570510716296383 

698/698 [==============================] - 196s 281ms/step - loss: 1.5405
Epoch 16/20
698/698 [==============================] - ETA: 0s - loss: 1.5351
 mean perplexity: 4.529321752008171 

698/698 [==============================] - 191s 274ms/step - loss: 1.5351
Epoch 17/20
698/698 [==============================] - ETA: 0s - loss: 1.5295
 mean perplexity: 4.5790521657984 

698/698 [==============================] - 184s 264ms/step - loss: 1.5295
Epoch 18/20
698/698 [==============================] - ETA: 0s - loss: 1.5245
 mean perplexity: 4.504259900569355 

Stopping training...
698/698 [=====

### 3. Predicción del próximo caracter

In [52]:
import gradio as gr

def model_response(human_text):

    # Encodeamos
    encoded = [char2idx[ch] for ch in human_text.lower() ]
    # Si tienen distinto largo
    encoded = pad_sequences([encoded], maxlen=max_context_size, padding='pre')

    # Predicción softmax
    y_hat = np.argmax(model.predict(encoded)[0,-1,:])


    # Debemos buscar en el vocabulario el caracter
    # que corresopnde al indice (y_hat) predicho por le modelo
    out_word = ''
    out_word = idx2char[y_hat]

    # Agrego la palabra a la frase predicha
    return human_text + out_word

iface = gr.Interface(
    fn=model_response,
    inputs=["textbox"],
    outputs="text")

iface.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 15ms/step
Keyboard interruption in main thread... closing server.


In [41]:
# Se puede usar gradio para probar el modelo
# Gradio es una herramienta muy útil para crear interfaces para ensayar modelos
# https://gradio.app/

!pip install -q gradio

### 4. Generador de secuencias

In [45]:
def generate_seq(model, seed_text, max_length, n_words):
    """
        Exec model sequence prediction

        Args:
            model (keras): modelo entrenado
            seed_text (string): texto de entrada (input_seq)
            max_length (int): máxima longitud de la sequencia de entrada
            n_words (int): números de caracteres a agregar a la sequencia de entrada
        returns:
            output_text (string): sentencia con las "n_words" agregadas
    """
    output_text = seed_text
	# generate a fixed number of words
    for _ in range(n_words):
		# Encodeamos
        encoded = [char2idx[ch] for ch in output_text.lower() ]
		# Si tienen distinto largo
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

		# Predicción softmax
        y_hat = np.argmax(model.predict(encoded,verbose=0)[0,-1,:])
		# Vamos concatenando las predicciones
        out_word = ''

        out_word = idx2char[y_hat]

		# Agrego las palabras a la frase predicha
        output_text += out_word
    return output_text

In [46]:
input_text='add salt to the'

generate_seq(model, input_text, max_length=max_context_size, n_words=30)

'add salt to the pasta into the pasta into the'

### 5. Beam search y muestreo aleatorio

In [47]:
# funcionalidades para hacer encoding y decoding

def encode(text,max_length=max_context_size):

    encoded = [char2idx[ch] for ch in text]
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

    return encoded

def decode(seq):
    return ''.join([idx2char[ch] for ch in seq])

In [48]:
from scipy.special import softmax

# función que selecciona candidatos para el beam search
def select_candidates(pred,num_beams,vocab_size,history_probs,history_tokens,temp,mode):

  # colectar todas las probabilidades para la siguiente búsqueda
  pred_large = []

  for idx,pp in enumerate(pred):
    pred_large.extend(np.log(pp+1E-10)+history_probs[idx])

  pred_large = np.array(pred_large)

  # criterio de selección
  if mode == 'det':
    idx_select = np.argsort(pred_large)[::-1][:num_beams] # beam search determinista
  elif mode == 'sto':
    idx_select = np.random.choice(np.arange(pred_large.shape[0]), num_beams, p=softmax(pred_large/temp)) # beam search con muestreo aleatorio
  else:
    raise ValueError(f'Wrong selection mode. {mode} was given. det and sto are supported.')

  # traducir a índices de token en el vocabulario
  new_history_tokens = np.concatenate((np.array(history_tokens)[idx_select//vocab_size],
                        np.array([idx_select%vocab_size]).T),
                      axis=1)

  # devolver el producto de las probabilidades (log) y la secuencia de tokens seleccionados
  return pred_large[idx_select.astype(int)], new_history_tokens.astype(int)


def beam_search(model,num_beams,num_words,input,temp=1,mode='det'):

    # first iteration

    # encode
    encoded = encode(input)

    # first prediction
    y_hat = model.predict(encoded,verbose=0)[0,-1,:]

    # get vocabulary size
    vocab_size = y_hat.shape[0]

    # initialize history
    history_probs = [0]*num_beams
    history_tokens = [encoded[0]]*num_beams

    # select num_beams candidates
    history_probs, history_tokens = select_candidates([y_hat],
                                        num_beams,
                                        vocab_size,
                                        history_probs,
                                        history_tokens,
                                        temp,
                                        mode)

    # beam search loop
    for i in range(num_words-1):

      preds = []

      for hist in history_tokens:

        # actualizar secuencia de tokens
        input_update = np.array([hist[i+1:]]).copy()

        # predicción
        y_hat = model.predict(input_update,verbose=0)[0,-1,:]

        preds.append(y_hat)

      history_probs, history_tokens = select_candidates(preds,
                                                        num_beams,
                                                        vocab_size,
                                                        history_probs,
                                                        history_tokens,
                                                        temp,
                                                        mode)

    return history_tokens[:,-(len(input)+num_words):]

#### Predicción

In [53]:
inputs = ["add salt to the","put the chicken in the", "heat the oil in a", "add the flour to the", "mix the eggs with the"]

for input in inputs:
  print(f'Input: {input}')
  salida = beam_search(model,num_beams=10,num_words=6,input=input,temp=1,mode='sto')
  print(f'Ouput:{decode(salida[0])}')

  print ('----------------------------------')


Input: add salt to the
Ouput:add salt to the oven 
----------------------------------
Input: put the chicken in the
Ouput:put the chicken in the oven 
----------------------------------
Input: heat the oil in a
Ouput:heat the oil in a large
----------------------------------
Input: add the flour to the
Ouput:add the flour to the sauce
----------------------------------
Input: mix the eggs with the
Ouput:mix the eggs with the sauce
----------------------------------


# Conclusiones finales
1. Encontré que el costo computacional del entrenamiento del modelo es muy alto. Considerando que limité bastante el número de recetas, en un ambiente productivo, no lo veo óptimo para su uso. Esto se repite tanto como en el modelo de word como este, aunque aquí, al procesar por caracteres, se refuerza esta problemática.

2. En cuanto a la predicción de secuencias, noto que las primeras palabras tienen sentido, no así si intentamos predecir una cadena más larga, en donde, a pesar de encontrar palabras del idioma inglés (las recetas están escritas en dicho idioma), se va perdiendo el sentido de la oración, por ej: "add salt to the pasta into the pasta into the".

3. Utilizando beam search y muestreo aleatorio, y a pesar de la simplesa del algoritmo y que limité el número de recetas, encuentro que las secuencia sugerencias tienen un léxico coherente, en cuanto a la semántica, tal vez falla en algunas, por ejemplo, uno no agrega sal dentro del horno, pero sí se lo hace con un pollo (predicción correcta encontrada por el modelo también)

4. Comparando el modelo por palabras y por caracter, considero que, aunque no con grandes diferencias, el modelo por caracter tienen mejor accuracy, analizándolo semánticamente, es decir revisando si la salida de las predicciones tienen sentido en el contexto de las recetas de cocina.

## Próximos pasos
Al igual que mencioné en el modelo por palabras, para realizar un mejor análisis, se podría usar Optuna u algún otro framework de evaluación de hiperparámetros, para ir "jugando" con los diferentes hiperparámetros, por una cuestión de tiempo de procesamiento y costo de GPU, no lo implementé, pero sí sería necesario en caso que en mi TP final quiera aplicar algo de esta temática.